In [ ]:
#install Skychart in the dir c:/ceil
#running this notebook will then repace the solar graphic with a recient image from internet.
#Several possibilities are listed .

import matplotlib.pyplot as plt
import requests # request img from web
import shutil # save img locally
import cv2
import numpy as np
import time
time.strftime("%B %d, %Y")
#pip install pymeeus

import pymeeus#https://www.amazon.com/exec/obidos/ASIN/0943396352/qid=1096827202/sr=2-1/ref=pd_ka_2_1/104-5453370-8789522
#C:\Users\Mike\anaconda3\Lib\site-packages\pymeeus\__pycache__
from pymeeus.Angle import Angle
from pymeeus.base import TOL
from pymeeus.Epoch import Epoch
from datetime import datetime
from pymeeus.Sun import Sun

def img_is_color(img):

    if len(img.shape) == 3:
        # Check the color channels to see if they're all the same.
        c1, c2, c3 = img[:, : , 0], img[:, :, 1], img[:, :, 2]
        if (c1 == c2).all() and (c2 == c3).all():
            return True

    return False

def show_image_list(list_images, list_titles=None, list_cmaps=None, grid=True, num_cols=2, figsize=(20, 10), title_fontsize=30):
    '''
    Shows a grid of images, where each image is a Numpy array. The images can be either
    RGB or grayscale.

    Parameters:
    ----------
    images: list
        List of the images to be displayed.
    list_titles: list or None
        Optional list of titles to be shown for each image.
    list_cmaps: list or None
        Optional list of cmap values for each image. If None, then cmap will be
        automatically inferred.
    grid: boolean
        If True, show a grid over each image
    num_cols: int
        Number of columns to show.
    figsize: tuple of width, height
        Value to be passed to pyplot.figure()
    title_fontsize: int
        Value to be passed to set_title().
    '''

    assert isinstance(list_images, list)
    assert len(list_images) > 0
    assert isinstance(list_images[0], np.ndarray)

    if list_titles is not None:
        assert isinstance(list_titles, list)
        assert len(list_images) == len(list_titles), '%d imgs != %d titles' % (len(list_images), len(list_titles))

    if list_cmaps is not None:
        assert isinstance(list_cmaps, list)
        assert len(list_images) == len(list_cmaps), '%d imgs != %d cmaps' % (len(list_images), len(list_cmaps))

    num_images  = len(list_images)
    num_cols    = min(num_images, num_cols)
    num_rows    = int(num_images / num_cols) + (1 if num_images % num_cols != 0 else 0)

    # Create a grid of subplots.
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    
    # Create list of axes for easy iteration.
    if isinstance(axes, np.ndarray):
        list_axes = list(axes.flat)
    else:
        list_axes = [axes]

    for i in range(num_images):

        img    = list_images[i]
        title  = list_titles[i] if list_titles is not None else 'Image %d' % (i)
        cmap   = list_cmaps[i] if list_cmaps is not None else (None if img_is_color(img) else 'gray')
        
        list_axes[i].imshow(img, cmap=cmap)
        list_axes[i].set_title(title, fontsize=title_fontsize) 
        list_axes[i].grid(grid)

    for i in range(num_images, len(list_axes)):
        list_axes[i].set_visible(False)

    fig.tight_layout()
    _ = plt.show()
current_month = datetime.now().strftime('%m') # 02 This is 0 padded
current_month_text = datetime.now().strftime('%h') # Feb
current_month_text = datetime.now().strftime('%B') # February

current_day = datetime.now().strftime('%d')   # 23 This is also padded
current_day_text = datetime.now().strftime('%a')  # Fri
current_day_full_text = datetime.now().strftime('%A')  # Friday

current_weekday_day_of_today = datetime.now().strftime('%w') # 5  Where 0 is Sunday and 6 is Saturday.

current_year_full = datetime.now().strftime('%Y')  # 2018
current_year_short = datetime.now().strftime('%y')  # 18 without century

current_second= datetime.now().strftime('%S') # 53
current_minute = datetime.now().strftime('%M') # 38
current_hour = datetime.now().strftime('%H') # 16 like 4pm
current_hour = datetime.now().strftime('%I') # 04 pm

current_hour_am_pm = datetime.now().strftime('%p') # 4 pm

current_microseconds = datetime.now().strftime('%f') # 623596 Rarely we need.

current_timzone = datetime.now().strftime('%Z') # UTC, EST, CST etc. (empty string if the object is naive).

def SunPangle():#pymeeus
    current_year = int(datetime.now().strftime('%Y'))  # 2018
    current_month = int(datetime.now().strftime('%m')) # 02 This is 0 padded
    current_day = int(datetime.now().strftime('%d'))   # 23 This is also padded
    epoch = Epoch(current_year, current_month, current_day)

    p, b0, l0 = Sun.ephemeris_physical_observations(epoch)

    print("Ephemeris of physical observations of the Sun:")

    print("P ", round(p, 2))

    # P : 26.27

    print("B0", round(b0, 2))

    # B0: 5.99

    print("L0", round(l0, 2))

    # L0: 238.63
    return p

def rotate_about_point(x, y, degree, image):# rotating about an arbitrary point (x,y) using only openCV https://stackoverflow.com/questions/9041681/opencv-python-rotate-image-by-x-degrees-around-specific-point
    rot_mtx = cv.getRotationMatrix2D((x, y), angle, 1)
    abs_cos = abs(rot_mtx[0, 0])
    abs_sin = abs(rot_mtx[0, 1])
    rot_wdt = int(frm_hgt * abs_sin + frm_wdt * abs_cos)
    rot_hgt = int(frm_hgt * abs_cos + frm_wdt * abs_sin)
    rot_mtx += np.asarray([[0, 0, -lftmost_x],
                           [0, 0, -topmost_y]])
    rot_img = cv.warpAffine(image, rot_mtx, (rot_wdt, rot_hgt),
                            borderMode=cv.BORDER_CONSTANT)
    return rot_img
    
def rotateImage(image, angle):
    center=tuple(np.array(image.shape[0:2])/2)
    print ("angleP",angle)
    rot_mat = cv2.getRotationMatrix2D(center,float(angle),1.0)
    return cv2.warpAffine(image, rot_mat, image.shape[0:2],flags=cv2.INTER_LINEAR)
    

def funcRotate(degree=0):
    degree = cv2.getTrackbarPos('degree','Frame')
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), degree, 1)
    rotated_image = cv2.warpAffine(original, rotation_matrix, (width, height))
    #img_scaled = cv2.normalize(img, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)
    cv2.imshow('Rotate', rotated_image)
    return rotated_image
   
def extract_floats(string):
    """Extracts floating-point numbers from a string.

    Args:
        string: The input string.

    Returns:
        A list of extracted floating-point numbers.
    """

    pattern = r"[-+]?\d+\.\d+"
    matches = re.findall(pattern, string)
    return [float(match) for match in matches]
    
def add_padding_to_image(img, padding_width):#https://samruddhichitnis02.medium.com/how-to-zero-pad-images-from-scratch-using-numpy-d8779abf44d5
    img_with_padding = np.zeros([(img.shape[0] + padding_width * 2), (img.shape[0] + padding_width * 2), 3])
    
    r_pad = img_with_padding[:, :, 0]
    g_pad = img_with_padding[:, :, 1]
    b_pad = img_with_padding[:, :, 2]
    
    r_pad[padding_width : -padding_width, padding_width : -padding_width] = img[:, :, 0]
    g_pad[padding_width : -padding_width, padding_width : -padding_width] = img[:, :, 1]
    b_pad[padding_width : -padding_width, padding_width : -padding_width] = img[:, :, 2]
    
    fin_arr = np.dstack((r_pad, g_pad, b_pad))
    return fin_arr
file_name = "C:\Ciel\data\planet\sun-0.jpg" ##input('Save image as (string):') #prompt user for file name

#s = "https://spaceweather.com/images" + time.strftime("%Y") + "/" + time.strftime("%d") + time.strftime("%B")[0:3].lower() + time.strftime("%Y")[2:4] + "/hmi1898.gif" 
#s = "https://umbra.nascom.nasa.gov/images/latest_aia_94.gif"
#s = "https://umbra.nascom.nasa.gov/images/latest_aia_1700.gif"
#s = "https://umbra.nascom.nasa.gov/images/latest_hmi_igram.gif"
s = "https://umbra.nascom.nasa.gov/images/latest_aia_171.gif"
s = "https://www.spaceweatherlive.com/images/SDO/SDO_HMIIF_512.jpg"; pad=30
s = "http://cesar.esa.int/sun_monitor/image_hel_halpha_large_latest.jpg"; pad=73
#res = C:\Users\Mike\Pictures\limb2412311241-512x512.png

print (s)

res = requests.get(s, stream = True)#https://www.scrapingbee.com/blog/download-image-python/

#res = "C:\\Users\\Mike\\Pictures\\limb2412311241-512x512.png"

if res.status_code == 200:
    with open(file_name,'wb') as f:
        shutil.copyfileobj(res.raw, f)
    print('Image sucessfully Downloaded: ',file_name)
    img1 = cv2.imread(file_name)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    #print(type(img))
    #cv2.imshow("as downloaded", img)
    #cv2.waitKey(1)
    #plt.imshow(img1)
    p= SunPangle()
    img2=rotateImage(img1, p)#http://www.jgiesen.de/sunrot/index.html
    #img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    #print(type(img))
    #cv2.imshow("rotated P angle", img)
    #cv2.waitKey(1)
    #plt.imshow(img1)
    height, width = img2.shape[:2]
    print("height ",height,"width ",width)
    height=int(height*2)
    width=int(width*2)
    print("height ",height,"width ",width)
    
    #res = cv2.resize(img, dsize=(48,48), interpolation=cv2.INTER_CUBIC)
    img3 = add_padding_to_image(img2, pad)
    img3 = cv2.normalize(img3, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
    
    height, width = img3.shape[:2]
    print("height ",height,"width ",width)
    #print(type(res))
    #cv2.imshow("padded for saling in Sky Chart", res)
    #cv2.waitKey(1)
    img4 = cv2.normalize(img3, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
    img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
    #plt.imshow(res)
    
    
    result = cv2.imwrite("C:\Ciel\data\planet\sun-0.jpg",img4)
    print("result",result)
    show_image_list(list_images=[img1, img2, img3, img4], 
                list_titles=['img1', 'img2', 'img3', 'img4'],
                num_cols=4,
                figsize=(20, 10),
                grid=False,
                title_fontsize=20)
else:
    print('Image Couldn\'t be retrieved')